# 🤖 Vector Chatbot with Memory

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Sayandip05/vector_chatbot/blob/main/VECTOR_CHATBOT.ipynb)

Complete implementation using ChromaDB and sentence transformers

In [ ]:
# ========================================
# STEP 1: INSTALL DEPENDENCIES
# ========================================
print("📦 Installing dependencies...")
!pip -q install chromadb sentence-transformers transformers accelerate einops torch
print("✅ Installation complete!\n")

In [ ]:
# ========================================
# STEP 2: IMPORT LIBRARIES
# ========================================
print("📚 Importing libraries...")

import os
import time
import uuid
import json
from dataclasses import dataclass, asdict
from typing import List, Dict, Optional, Tuple

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer

print("✅ Libraries imported!\n")

In [ ]:
# ========================================
# STEP 3: CONFIGURATION
# ========================================
print("⚙️ Setting up configuration...")

# Model Settings
CHAT_MODEL = "Qwen/Qwen2.5-0.5B-Instruct"
EMBED_MODEL = "all-MiniLM-L6-v2"

# Database Settings
PERSIST_DIR = "/content/memory_db"
COLLECTION_NAME = "chat_memory"

# Chat Settings
MAX_NEW_TOKENS = 256
TEMPERATURE = 0.7
TOP_K_RETRIEVAL = 6
CONTEXT_CHAR_LIMIT = 1600

# User Settings
USER_ID = "colab_user"
CONVERSATION_ID = "conv_001"

# System Prompt
SYSTEM_PROMPT = (
    "You are a helpful and friendly assistant. "
    "When you remember something from past conversations, mention it naturally. "
    "Keep responses concise and helpful."
)

# Create persist directory
os.makedirs(PERSIST_DIR, exist_ok=True)
print(f"✅ Configuration ready! Database: {PERSIST_DIR}\n")

In [ ]:
# ========================================
# STEP 4: EMBEDDING HANDLER CLASS
# ========================================
print("🔢 Creating Embedding Handler...")

class EmbeddingHandler:
    """Converts text to vector embeddings"""

    def __init__(self, model_name: str = EMBED_MODEL):
        print(f"  Loading embedding model: {model_name}...")
        self.model = SentenceTransformer(model_name)
        print(f"  ✅ Embedding model loaded!")

    def embed_texts(self, texts: List[str]) -> List[List[float]]:
        """Convert texts to embeddings"""
        vectors = self.model.encode(
            texts,
            convert_to_numpy=True,
            normalize_embeddings=True
        )
        return [vec.tolist() for vec in vectors]

    def embed_single(self, text: str) -> List[float]:
        """Convert single text to embedding"""
        return self.embed_texts([text])[0]

print("✅ Embedding Handler ready!\n")

# ========================================
# STEP 5: MEMORY ITEM CLASS
# ========================================

@dataclass
class MemoryItem:
    """Single memory entry"""
    id: str
    role: str
    content: str
    user_id: str
    conversation_id: str
    created_at: float

In [ ]:
# ========================================
# STEP 6: MEMORY MANAGER CLASS
# ========================================
print("💾 Creating Memory Manager...")

class MemoryManager:
    """Manages conversation memory"""

    def __init__(self):
        print("  Initializing ChromaDB...")

        # Create embedder
        self.embedder = EmbeddingHandler()

        # Setup ChromaDB
        self.client = chromadb.PersistentClient(
            path=PERSIST_DIR,
            settings=Settings(anonymized_telemetry=False)
        )

        self.collection = self.client.get_or_create_collection(
            name=COLLECTION_NAME,
            metadata={"hnsw:space": "cosine"}
        )

        print(f"  ✅ Memory ready! Stored items: {self.collection.count()}")

    def add(self, items: List[MemoryItem]):
        """Add memories"""
        if not items:
            return

        texts = [item.content for item in items]
        embeddings = self.embedder.embed_texts(texts)

        self.collection.add(
            ids=[item.id for item in items],
            documents=texts,
            metadatas=[asdict(item) for item in items],
            embeddings=embeddings
        )

    def retrieve(self, query: str, top_k: int = 6) -> List[Tuple[str, Dict, float]]:
        """Retrieve relevant memories"""
        query_embedding = self.embedder.embed_single(query)

        results = self.collection.query(
            query_embeddings=[query_embedding],
            n_results=top_k,
            where={"user_id": USER_ID},
            include=["documents", "metadatas", "distances"]
        )

        docs = results.get("documents", [[]])[0]
        metas = results.get("metadatas", [[]])[0]
        dists = results.get("distances", [[]])[0]

        return list(zip(docs, metas, dists))

    def build_context(self, retrieved: List[Tuple[str, Dict, float]]) -> str:
        """Build context from memories"""
        if not retrieved:
            return "(no relevant memory)"

        chunks = []
        used_chars = 0

        for i, (doc, meta, dist) in enumerate(retrieved, start=1):
            chunk = f"[memory {i}] {meta.get('role')}: {doc.strip()}\n"

            if used_chars + len(chunk) > CONTEXT_CHAR_LIMIT:
                break

            chunks.append(chunk)
            used_chars += len(chunk)

        return "\n".join(chunks) if chunks else "(no relevant memory)"

    def create_memory_item(self, role: str, content: str) -> MemoryItem:
        """Create new memory"""
        return MemoryItem(
            id=str(uuid.uuid4()),
            role=role,
            content=content,
            user_id=USER_ID,
            conversation_id=CONVERSATION_ID,
            created_at=time.time()
        )

    def count(self) -> int:
        """Get memory count"""
        return self.collection.count()

    def search(self, query: str, top_k: int = 5) -> List[Dict]:
        """Search memories"""
        results = self.retrieve(query, top_k=top_k)

        formatted = []
        for doc, meta, dist in results:
            formatted.append({
                "role": meta.get("role"),
                "content": doc,
                "distance": round(dist, 3),
                "timestamp": time.strftime("%Y-%m-%d %H:%M:%S",
                                          time.localtime(meta.get("created_at", 0)))
            })

        return formatted

print("✅ Memory Manager ready!\n")

In [ ]:
# ========================================
# STEP 7: CHATBOT CLASS
# ========================================
print("🤖 Creating Chatbot...")

class Chatbot:
    """Main chatbot with memory"""

    def __init__(self):
        print("\n" + "="*50)
        print("  INITIALIZING CHATBOT")
        print("="*50 + "\n")

        # Initialize memory
        self.memory = MemoryManager()

        # Load chat model
        print(f"  Loading chat model: {CHAT_MODEL}...")
        self.tokenizer = AutoTokenizer.from_pretrained(CHAT_MODEL)

        self.model = AutoModelForCausalLM.from_pretrained(
            CHAT_MODEL,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto"
        )

        self.pipeline = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer
        )

        print("  ✅ Chat model loaded!")
        print("\n" + "="*50)
        print("  ✅ CHATBOT READY!")
        print("="*50 + "\n")

    def generate_response(self, messages: List[Dict[str, str]]) -> str:
        """Generate model response"""
        prompt = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        output = self.pipeline(
            prompt,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=True,
            temperature=TEMPERATURE,
            top_p=0.9,
            top_k=50
        )

        generated_text = output[0]["generated_text"][len(prompt):]
        return generated_text.strip()

    def chat(self, user_message: str) -> str:
        """Main chat function"""
        # Retrieve context
        retrieved = self.memory.retrieve(user_message, top_k=TOP_K_RETRIEVAL)
        context = self.memory.build_context(retrieved)

        # Build messages
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {
                "role": "user",
                "content": f"PAST CONTEXT:\n{context}\n\nQUESTION: {user_message}"
            }
        ]

        # Generate response
        response = self.generate_response(messages)

        # Store in memory
        user_item = self.memory.create_memory_item("user", user_message)
        assistant_item = self.memory.create_memory_item("assistant", response)
        self.memory.add([user_item, assistant_item])

        return response

    def search_memory(self, query: str, top_k: int = 5) -> List[Dict]:
        """Search conversation history"""
        return self.memory.search(query, top_k=top_k)

    def memory_count(self) -> int:
        """Get total memories"""
        return self.memory.count()

    def seed_memory(self, conversations: List[tuple]):
        """Seed initial memories"""
        items = []
        for role, content in conversations:
            items.append(self.memory.create_memory_item(role, content))

        self.memory.add(items)
        print(f"✓ Seeded {len(items)} memories\n")

print("✅ Chatbot class ready!\n")

In [ ]:
# ========================================
# STEP 8: INITIALIZE CHATBOT
# ========================================
print("🚀 Starting chatbot initialization...\n")

bot = Chatbot()

In [ ]:
# ========================================
# STEP 9: SEED INITIAL MEMORIES
# ========================================
print("🌱 Seeding initial memories...")

bot.seed_memory([
    ("user", "My name is Alex"),
    ("assistant", "Nice to meet you, Alex!"),
    ("user", "I prefer Python examples over JavaScript"),
    ("assistant", "Got it! I'll use Python for examples."),
    ("user", "I'm learning machine learning"),
    ("assistant", "That's great! ML is fascinating!")
])

In [ ]:
# ========================================
# STEP 10: INTERACTIVE CHAT FUNCTION
# ========================================

def chat_with_bot():
    """Interactive chat function"""
    print("\n" + "="*60)
    print("🎯 CHATBOT STARTED!")
    print("="*60)
    print("Commands:")
    print("  • Just type to chat")
    print("  • 'search: <query>' - Search memory")
    print("  • 'count' - Show total memories")
    print("  • 'exit' - Stop chatting")
    print("="*60 + "\n")

    while True:
        try:
            user_input = input("You: ").strip()

            if not user_input:
                continue

            # Exit
            if user_input.lower() in ["exit", "quit", "stop"]:
                print("\n👋 Goodbye!\n")
                break

            # Memory count
            if user_input.lower() == "count":
                count = bot.memory_count()
                print(f"📊 Total memories: {count}\n")
                continue

            # Memory search
            if user_input.lower().startswith("search:"):
                query = user_input[7:].strip()
                if query:
                    print(f"\n🔍 Searching for: '{query}'")
                    results = bot.search_memory(query, top_k=3)

                    if results:
                        for i, result in enumerate(results, 1):
                            print(f"\n[{i}] {result['role']} (similarity: {result['distance']})")
                            print(f"    Time: {result['timestamp']}")
                            print(f"    Content: {result['content']}")
                    else:
                        print("No results found.")
                    print()
                continue

            # Regular chat
            print("Assistant: ", end="", flush=True)
            response = bot.chat(user_input)
            print(response + "\n")

        except KeyboardInterrupt:
            print("\n\n👋 Goodbye!\n")
            break
        except Exception as e:
            print(f"❌ Error: {e}\n")

In [ ]:
# ========================================
# STEP 11: DEMO CONVERSATIONS
# ========================================

print("\n" + "="*60)
print("📝 DEMO CONVERSATIONS")
print("="*60 + "\n")

demo_questions = [
    "What's my name?",
    "What programming language do I prefer?",
    "What am I learning?",
]

for question in demo_questions:
    print(f"You: {question}")
    response = bot.chat(question)
    print(f"Assistant: {response}\n")

In [ ]:
# ========================================
# STEP 12: START INTERACTIVE CHAT
# ========================================

# Uncomment to start interactive chat
# chat_with_bot()

# Or use these commands directly:
# response = bot.chat("Your message here")
# print(response)
#
# results = bot.search_memory("query")
# for r in results:
#     print(r)
#
# print(f"Total memories: {bot.memory_count()}")